(weight-proportions)=
# Compute age- and length-binned weight proportions

In [ ]:
%run ./number_proportions.ipynb

## Import necessary modules

With the biological data loaded and number proportions computed, we can now compute the weight proportions. The first step is to distribute binned weights over age, length, and sex across strata. 

## Distributed weights over age, length, and sex across strata

The binned weight distributions are computed over values specific to aged (`dict_df_bio["specimen"]`) and unaged fish (`dict_df_bio["length"]`). This is done using the `binned_weights` function from the `survey.proportions` module and is done separately since the two datasets are processed somewhat differently before being combined in a later step. Here, we will use `"stratum_ks"` as our stratification definition.

In [2]:
# Pre-allocate a dictionary
ds_da_weight_dist = xr.Dataset()

# Aged
ds_da_weight_dist["aged"] = get_proportions.binned_weights(
    length_data=dict_df_bio["specimen"],
    include_filter={"sex": ["female", "male"]},
    interpolate_regression=False,
    group_columns=["stratum_ks", "sex", "age_bin"],
)

# Unaged
ds_da_weight_dist["unaged"] = get_proportions.binned_weights(
    length_data=dict_df_bio["length"],
    include_filter={"sex": ["female", "male"]},
    interpolate_regression=True,
    length_weight_data=da_binned_weight_table,
    group_columns=["stratum_ks", "sex"],
)


Just like the binned counts, this produces an `xarray.Dataset` comprising the aged and unaged data arrays. 

## Convert summed weights into weight proportions

These binned weights are normalized into proportions using the `weight_proportions` function. However, there is a slightly different treatment done here due to the aforementioned separate processing of aged and unaged samples. So we first compute the weight proportions for aged-only animals, where we are using the summed *combined* weights (i.e. from the individual specimens and haul weight totals) to compute the proportions. This effectively represents the same quantity as `proportion_overall` present in the number proportions `xarray.Dataset` dictionary. 

In [ ]:
# Initialize Dictionary container
dict_ds_weight_proportion = {}

# Aged
dict_ds_weight_proportion["aged"] = get_proportions.weight_proportions(
    weight_data=ds_da_weight_dist["aged"], 
    catch_data=dict_df_bio["catch"], 
    group_columns = ["stratum_ks"]
)

This outputs an `xarray.Dataset` with the variable `"proportion_overall"` that is stored in the container dictionary `dict_da_weight_proportion`.

### Scaling weight proportions for unaged fish

The first step for the unaged weight proportions involves scaling the fitted binned weights from unaged fish, which requires a few additional steps. It incorporates the aged and unaged number proportions to estimate sex ratios, the binned weights computed for all fish, and the total catch weights.

In [ ]:
# Compute the scaled weight proportions for unaged fish
dict_ds_weight_proportion["unaged"] = get_proportions.fitted_weight_proportions(
    weight_data=ds_da_weight_dist["unaged"],
    reference_weight_proportions=dict_ds_weight_proportion["aged"],
    catch_data=dict_df_bio["catch"],
    number_proportions=dict_ds_number_proportion["unaged"],
    binned_weights=da_binned_weights_all,
    group_columns=["stratum_ks"]
)